In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVR
from sklearn import svm
import pickle as pk

In [68]:
df=pd.read_csv("TV_Laayoune_Data_CIAUMED.csv")

In [69]:
df

,Unnamed: 0,Program,Genre,Number of viewers,Audience share (%),Start time,Broadcasting date
0,0.0,Hiraf,Capsule,163000,22.0,21:15:00,2023-03-21
1,1.0,Midadou chouaara,Culturel,868000,13.8,06:17:00,2023-03-20
2,2.0,Fada Ryadi,Sport,681000,9.7,07:10:00,2023-03-19
3,3.0,Journal Télévisé,Info,615000,44.9,22:30:00,2023-03-18
4,4.0,Agenda,Info,868000,36.8,19:39:00,2023-03-17
...,...,...,...,...,...,...,...
5135,NaN,Amnir,documentaires,10344,48.0,15:31:17,2021-12-03
5136,NaN,Kadaya ramadaniya,Religieux,9994,47.0,15:09:28,2021-05-30
5137,NaN,Hiraf,Capsule,9299,30.0,13:45:15,2021-07-30
5138,NaN,Konoz mina sahrae,Culturel,9669,10.0,07:24:00,2022-10-25


In [70]:
df.dtypes

Unnamed: 0            float64
Program                object
Genre                  object
Number of viewers       int64
Audience share (%)    float64
Start time             object
Broadcasting date      object
dtype: object

In [71]:
df["Broadcasting date"]=pd.to_datetime(df["Broadcasting date"])

In [72]:
df["Year"]=df["Broadcasting date"].dt.year
df["Month"]=df["Broadcasting date"].dt.month
df["Day"]=df["Broadcasting date"].dt.day

In [73]:
X_cat=df[["Program"]]

In [74]:
df["Start time"]=pd.to_datetime(df["Start time"])

In [75]:
df["Hour"]=df["Start time"].dt.hour
df["Minute"]=df["Start time"].dt.minute

In [76]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#cv=TfidfVectorizer()
#cv.fit_transform(X_cat)

In [77]:
X_cat=pd.get_dummies(X_cat)

In [78]:
X_num=df[["Year","Month","Day","Hour","Minute"]]

In [79]:
from sklearn.preprocessing import MinMaxScaler

In [80]:
#scaler=MinMaxScaler()
#X_num=scaler.fit_transform(X_num)

In [81]:
X=np.hstack([X_cat,X_num])

In [87]:
y=df["Audience share (%)"]

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=23,test_size=0.3)

In [90]:
y.value_counts()

27.0    197
21.0    195
20.0    189
38.0    189
23.0    185
30.0    183
24.0    178
26.0    176
35.0    173
22.0    170
25.0    169
31.0    169
32.0    167
39.0    167
28.0    166
34.0    165
36.0    165
37.0    164
29.0    153
19.0    152
17.0    140
11.0    137
14.0    137
33.0    135
18.0    132
10.0    126
13.0    121
15.0    118
16.0    111
12.0    107
41.0     58
40.0     44
54.0     24
42.0     21
43.0     20
55.0     19
51.0     18
53.0     17
59.0     17
46.0     17
52.0     17
45.0     16
49.0     16
57.0     16
58.0     15
56.0     15
50.0     14
44.0     13
48.0     11
47.0      5
19.8      4
12.5      1
13.8      1
36.8      1
9.7       1
14.8      1
36.3      1
44.9      1
Name: Audience share (%), dtype: int64

In [91]:
# Importing Models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Importing evaluation modules
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold, cross_val_score
# prepare the cross-validation procedure
cv = KFold(n_splits=5, random_state=1, shuffle=True)

# check the performance on diffrent regressor
models = []
models.append(('Ridge', Ridge()))
models.append(('LinearRegression', LinearRegression()))
models.append(('KNeighborsRegressor', KNeighborsRegressor()))
models.append(('Random Forest', RandomForestRegressor()))
models.append(('Decision Tree', DecisionTreeRegressor()))

train_l = []
test_l = []
mae_l = []
rmse_l = []
r2_l = []

import time
i = 0
for name,model in models:
    i = i+1
    start_time = time.time()
    
    # Fitting model to the Training set
    clf = model
    clf.fit(X_train, y_train)
    
    # Scores of model
    train = model.score(X_train, y_train)
    test = model.score(X_test, y_test)
    
    train_l.append(train)
    test_l.append(test)
    
    # predict values
    predictions = clf.predict(X_test)
    # RMSE
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    rmse_l.append(rmse)
    # MAE
    mae = mean_absolute_error(y_test,predictions)
    mae_l.append(mae)
    # R2 score
    r2 = r2_score(y_test,predictions)
    r2_l.append(r2)
    scores = cross_val_score(clf, X, y, cv=cv, n_jobs=-1)


    print("+","="*100,"+")
    print('\033[1m' + f"\t\t\t{i}-For {name} The Performance result is: " + '\033[0m')
    print("+","="*100,"+")
    print('Root mean squared error (RMSE) : ', rmse)   
    print("-"*50)
    print('Mean absolute error (MAE) : ', mae)
    print("-"*50)
   # print('Max errors : ', m_errors)
    print("-"*50)
    print('R2 score : ', r2)
    print("-"*50)
    print('cross validation accuracy : ', np.mean(scores))
    print("-"*50)


    
    print("\t\t\t\t\t\t\t-----------------------------------------------------------")
    print(f"\t\t\t\t\t\t\t Time for detection ({name}) : {round((time.time() - start_time), 3)} seconds...")
    print("\t\t\t\t\t\t\t-----------------------------------------------------------")
    print()
    
comp = pd.DataFrame({"Training Score": train_l, "Testing Score": test_l, "MAE": mae_l, "RMSE": rmse_l, "R2 Score": r2_l})
comp

+ ==================================================================================================== +
			1-For Ridge The Performance result is: 
+ ==================================================================================================== +
Root mean squared error (RMSE) :  10.20632212485911
--------------------------------------------------
Mean absolute error (MAE) :  8.368787491043514
--------------------------------------------------
--------------------------------------------------
R2 score :  -0.013917275560880071
--------------------------------------------------
cross validation accuracy :  -0.009253164495833976
--------------------------------------------------
							-----------------------------------------------------------
							 Time for detection (Ridge) : 0.204 seconds...
							-----------------------------------------------------------

+ ==================================================================================================== +
			2-For Linea

,Training Score,Testing Score,MAE,RMSE,R2 Score
0,0.008375,-0.013917,8.368787,10.206322,-0.013917
1,0.008475,-0.014228,8.370044,10.207884,-0.014228
2,0.196882,-0.199039,9.033722,11.099019,-0.199039
3,0.848614,-0.105827,8.731082,10.658878,-0.105827
4,0.999538,-1.110060,11.775227,14.723649,-1.110060
